In [1]:
from missingpy import MissForest
from utils import *
import numpy as np
from sklearn.impute import KNNImputer
from Bio.KEGG import REST
import time
import argparse
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter
sns.set()

def get_mask(bs, nb_genes, m_low=0.5, m_high=0.5):
    # Compute masks
    # m_low = 0.5?
    p_mask = np.random.uniform(low=m_low, high=m_high, size=(bs,))  # Probability of setting mask to 0
    mask = np.random.binomial(1, p_mask, size=(nb_genes, bs)).astype(np.float32).T  # Shape=(bs, nb_genes)

    return mask

def get_dummies(cat_covs):
    cat_covs_onehot = []
    for c in range(cat_covs.shape[-1]):
        col = cat_covs[:, c]
        dummies = np.eye(len(np.unique(col)))[col]
        cat_covs_onehot.append(dummies)
    cat_covs_onehot = np.concatenate(cat_covs_onehot, axis=-1)
    return cat_covs_onehot

def r2_scores(x_gt, x_pred, mask):
    mask_r = np.copy(mask)
    mask_r[mask_r == 1] = np.nan

    gene_means = np.mean(x_gt, axis=0)  # Shape=(nb_genes,)
    mask_r[:, gene_means == 0] = np.nan  # Discard genes with 0 variance
    ss_res = np.nansum((1-mask_r)*(x_gt - x_pred) ** 2, axis=0)
    ss_tot = np.nansum((1-mask_r)*(x_gt - gene_means) ** 2, axis=0)
    r_sq = 1 - ss_res/(ss_tot)
    return r_sq

###########################
#      KEGG PATHWAYS      #
###########################

GENES_FILE = '/local/scratch/rv340/hugo/genes.tsv'

def list_gene_symbols():
    df = pd.read_csv(GENES_FILE, delimiter='\t', header=0)
    return df['Approved symbol'].values


def list_KEGG_human_pathways():
    lines = REST.kegg_list('pathway', 'hsa').readlines()
    symbols = np.array([s.split('\t')[0].split(':')[-1] for s in lines])
    description = np.array([s.split('\t')[1].rstrip() for s in lines])
    return symbols, description

def get_pathway_info(pathway):
    pathway_file = REST.kegg_get(pathway).read()  # query and read each pathway

    # iterate through each KEGG pathway file, keeping track of which section
    # of the file we're in, only read the gene in each pathway
    current_section = None
    gene_symbols = set()
    diseases = set()
    drugs = set()
    for line in pathway_file.rstrip().split('\n'):
        section = line[:12].strip()  # section names are within 12 columns
        if not section == '':
            current_section = section

        if current_section == 'DISEASE':
            disease = line[12:].split(' ')[0]
            diseases.add(disease)
        elif current_section == 'DRUG':
            drug = line[12:].split(' ')[0]
            drugs.add(drug)
        elif current_section == 'GENE':
            try:
                gene_identifiers, gene_description = line[12:].split('; ')
                gene_id, gene_symbol = gene_identifiers.split()
                gene_symbols.add(gene_symbol)
            except ValueError:
                print('WARNING: No gene found in {}'.format(line[12:]))
            
    return gene_symbols, diseases, drugs


def human_pathway_disease_drug_data(gene_symbols, human_pathways, diseases=None, drugs=None):
    hp = human_pathways
    nb_genes = len(gene_symbols)
    nb_pathways = len(hp)
    
    genes_p = np.zeros((nb_genes, nb_pathways))
    
    diseases_p = None
    if diseases is not None:
        diseases_p = np.zeros((len(diseases), nb_pathways))
    if drugs_p is not None:
        drugs_p = np.zeros((len(drugs), nb_pathways))
    
    for i, p in enumerate(hp):
        gs, di, dr = get_pathway_info(p)
        
        # Store genes of the pathway
        idxs = np.argwhere(np.isin(gene_symbols, list(gs))).flatten()
        genes_p[idxs, i] = 1
        
        # Store diseases of the pathway
        if diseses is not None:
            idxs = np.argwhere(np.isin(diseases, list(di))).flatten()
            diseases_p[idxs, i] = 1
        
        # Store drugs of the pathway
        if drugs is not None:
            idxs = np.argwhere(np.isin(drugs, list(dr))).flatten()
            drugs_p[idxs, i] = 1
    
    return genes_p, diseases_p, drugs_p


def human_pathway_data(gene_symbols, human_pathways):
    hp = human_pathways
    nb_genes = len(gene_symbols)
    nb_pathways = len(hp)
    
    genes_p = np.zeros((nb_genes, nb_pathways))
    
    for i, p in enumerate(hp):
        gs, _, _ = get_pathway_info(p)
        
        # Store genes of the pathway
        idxs = np.argwhere(np.isin(gene_symbols, list(gs))).flatten()
        genes_p[idxs, i] = 1
    
    return genes_p

def load_genes_pathway(pathway, gene_symbols, hp_desc, genes_p):
    pathway_idx = np.flatnonzero(np.core.defchararray.find(hp_desc, pathway)!=-1)[0]
    pathway = hp_desc[pathway_idx]

    # Genes from selected pathway
    genes_from_selected_pathway = genes_p[:, pathway_idx]
    genes_pathway_idxs = np.argwhere(genes_from_selected_pathway)[:, 0]
    gps = gene_symbols[genes_pathway_idxs]
    return gps

/auto/homes/rv340/adversarial-gtex/gtexenv/lib/python3.5/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [11]:
# Load dataset
x, symbols, sampl_ids, tissues = load_gtex()
x = np.float32(x)

# Load metadata
df_metadata = gtex_metadata()
print(df_metadata.head())

# Process categorical metadata
cat_cols = ['SEX', 'COHORT']  # 'SEX', 'COHORT'
df_metadata[cat_cols] = df_metadata[cat_cols].astype('category')
# cat_map = df_metadata[cat_cols].cat.categories
# print('cat map', cat_map)
cat_dicts = [df_metadata[cat_col].cat.categories.values for cat_col in cat_cols]
df_metadata[cat_cols] = df_metadata[cat_cols].apply(lambda x: x.cat.codes)
cat_covs = df_metadata.loc[sampl_ids, cat_cols].values
tissues_dict_inv = np.array(list(sorted(set(tissues))))
tissues_dict = {t: i for i, t in enumerate(tissues_dict_inv)}
tissues = np.vectorize(lambda t: tissues_dict[t])(tissues)
cat_dicts.append(tissues_dict_inv)
cat_covs = np.concatenate((cat_covs, tissues[:, None]), axis=-1)
cat_covs = np.int32(cat_covs)
print('Cat covs: ', cat_covs.shape)
cat_covs = get_dummies(cat_covs)

# Process numerical metadata
num_cols = ['AGE']
num_covs = df_metadata.loc[sampl_ids, num_cols].values
num_covs = standardize(num_covs)
num_covs = np.float32(num_covs)

# Train/test split
x_train, x_test, sampl_ids_train, sampl_ids_test = split_train_test_v2(x, sampl_ids)
x_train = standardize(x_train)
x_test = standardize(x_test)
num_covs_train, num_covs_test, _, _ = split_train_test_v2(num_covs, sampl_ids)
cat_covs_train, cat_covs_test, _, _ = split_train_test_v2(cat_covs, sampl_ids)
x_train, x_val, _, sampl_ids_val = split_train_test_v2(x_train, sampl_ids_train, train_rate=0.8)
num_covs_train, num_covs_val, _, _ = split_train_test_v2(num_covs_train, sampl_ids_train, train_rate=0.8)
cat_covs_train, cat_covs_val, sampl_ids_train, sampl_ids_val = split_train_test_v2(cat_covs_train, sampl_ids_train, train_rate=0.8)

                A1BG     A2LD1       A2M     A2ML1    A4GALT      AAAS  \
GTEX-NL4W   0.415623 -0.496085 -0.811018 -0.496085 -0.031079  1.374065   
GTEX-13O3O  0.053586 -0.681229 -0.444875 -0.126741 -1.118748  0.109482   
GTEX-12ZZW -0.292021  1.102440  0.717650  0.591696  0.512776  1.744448   
GTEX-XK95   1.282401 -2.066852  1.353622  1.442962  1.883649  0.031758   
GTEX-QDVJ  -0.051706 -0.181889  0.650353 -0.490597  1.038647 -0.288197   

                AACS     AAED1     AAGAB      AAK1  ...      ZW10    ZWILCH  \
GTEX-NL4W   0.720732 -2.040281  1.274520 -0.082959  ...  0.661543  0.741026   
GTEX-13O3O -0.209333 -1.056810 -0.302340  0.725124  ... -0.904346  1.541986   
GTEX-12ZZW -1.071858  0.292021 -1.606755 -0.739737  ... -0.204660 -1.671644   
GTEX-XK95  -0.117946 -0.919804 -0.853391 -0.663975  ... -1.783434 -0.668639   
GTEX-QDVJ   0.363147  1.038647  0.757996  0.626590  ...  1.038647  0.610943   

               ZWINT      ZXDA      ZXDB      ZXDC    ZYG11B       ZYX  \
GTEX-N

In [12]:
# Load genes and human pathways
print('Loading KEGG pathways information ...')
gene_symbols = list_gene_symbols()
hp, hp_desc = list_KEGG_human_pathways()
genes_p = human_pathway_data(gene_symbols, hp)

Loading KEGG pathways information ...


In [14]:
# 'Longevity regulating pathway - Homo sapiens'
# 'Alzheimer'
# 'p53'
# 'rhythm'
idx = np.where(['Alzheimer' in p for p in hp_desc])[0][0]
gs = gene_symbols[np.where(genes_p[:, idx])[0]]
gene_idxs = [i for i, g in enumerate(symbols) if g in gs]

In [15]:
len(gene_idxs)

273

# Impute MissForest

In [ ]:
# Compute R2 scores on test set
r2s = []
mses = []
times = []
runs = 3
n_estimators = [1]
x_train_pathway = x_train[:, gene_idxs]
x_test_pathway = x_test[:, gene_idxs]
nb_test, nb_genes = x_test_pathway.shape

for nt in n_estimators:
    r2s_ = []
    mses_ = []
    times_ = []
    
    for it in range(runs):
        # Mask test set elements
        mask = get_mask(nb_test, nb_genes, m_low=0.5, m_high=0.5)
        mask_zeros = np.copy(mask)
        mask[mask == 0] = np.nan
        x_test_nan = x_test_pathway * mask

        # Build train+test dataset and include covariates
        x_pathway = np.concatenate((x_train_pathway, x_test_nan), axis=0)
        cc = np.concatenate((cat_covs_train, cat_covs_test), axis=0)
        nc = np.concatenate((num_covs_train, num_covs_test), axis=0)
        c = np.concatenate((cc, nc), axis=-1)
        x_pathway_ = np.concatenate((x_pathway, c), axis=-1)

        # Impute and check time
        time1 = time.time()
        imputer = MissForest(bootstrap=True, class_weight=None, copy=True,
           criterion=('mse', 'gini'), decreasing=False, max_depth=None,
           max_features='auto', max_iter=10, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, missing_values=np.nan, n_estimators=nt,
           n_jobs=10, oob_score=False, random_state=None, verbose=1,
           warm_start=False)
        imputer.fit(x_pathway_)
        print('Fit')
        x_imp = imputer.transform(x_pathway_)
        x_test_imp = x_imp[-nb_test:, :nb_genes]

        imp_mse = np.sum((1-mask_zeros)*(x_test_imp - x_test_pathway)**2)/np.sum(1-mask_zeros)
        r2 = r2_scores(x_test_pathway, x_test_imp, mask_zeros)
        t = time.time() - time1
        print('n={}, MSE={}, R2={}, time={:.2f}'.format(nt, imp_mse, np.nanmean(r2), t))
        
        mses_.append(imp_mse)
        r2s_.append(np.nanmean(r2))
        times_.append(t)
    
    mses.append(mses_)
    r2s.append(r2s_)
    times.append(times_)

Fit


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    4.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Paral

[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    6.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    4.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.1s finished
[Parallel(n_jobs=

[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    4.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    3.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    6.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.1s finished
[Parallel(n_jobs=

[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    4.9s finished
[Parallel(n_jobs=

...

[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    3.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.1s finished
[Parallel(n_jobs=

[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    4.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    6.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.6s finished
[Parallel(n_jobs=

[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.1s finished
[Parallel(n_jobs=

[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    6.2s finished
[Parallel(n_jobs=

[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    4.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    6.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    5.1s finished
[Parallel(n_jobs=

In [ ]:
times = np.array(times)
times = times / 3600

In [ ]:
np.mean(times,axis=-1)

In [ ]:
np.std(times,axis=-1)

In [ ]:
r2s

In [ ]:
mses